# Motivation and Other Submissions
Eda.ipynb and model.ipynb were my quick first model (score around 0.135)
- No feature engineering
- Sqrt of SalePrice
- Ordinal encoding for all categorical variables
- Grid search on XGBoost (single model)

This model2 is now inspired by reading some other Kaggle submissions

[Top 1% Solution w/ Data Leakage](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/discussion/83751)

Feature Engineering

1. Fill NA values (some data leakage here with using medians)
2. Fill NA numerics w/ 0
3. Transform numeric features with box-cox (data leakage here as well)
4. Summed and year features (ex. bathrooms = \sum(bathrooms))
5. Has features (has pool, has 2nd floor, etc.)

Additional Engineering

6. Add dummies for categorical variables
7. Remove columns where one value dominates
8. Remove outliers

Modelling

1. Ridge Regression (Robust Scalar)
2. Lasso Regression
3. Elastic Net Regression
4. Support Vector Regression
5. Gradient Boosting Regressor
6. Light GBM
7. XGBoost
8. StackingCVRegressor w/ XGBoost as meta regressor

Final prediction is Blend of all previous models (including Stacking Regressor as one of those models)

[Top 4% no Data Leakage](https://www.kaggle.com/code/miftahuladib/housing-price-regression-top-4?scriptVersionId=202452540)

Feature Engineering

1. Sum features (like baths, porcharea, rooms)
2. Year features (ex. transform to 2025 - yearbuilt)
3. Fill NA Values (simply fill with 0 or 'No')

Additional Engineering

1. Remove Outliers (based on scatterplots of various numerical features)
2. Column Transformer (numeric --> standardScalar, ordinal --> ordinalEncoder, categorical --> oneHotEncoded)

Modelling

1. Random forest regressor
2. XGBoost
3. Ridge regression
4. Light GBM
5. CatBoost
6. VotingRegressor (not used)
7. StackingRegressor

# Goals and Plan

Goal: Understand which components of the other submissions are most relevant (how they affect score)

These components include:

1. Feature Engineering (which new features are best: summed features, year features, has features)
2. Feature Transformation and Filling (boxcox transformation, fill na with null vs. values)
3. Data leakage (how much does it help)

4. Column Transformers (scaling, ordinal transform vs. one hot encoding)
5. Removing outliers

6. Model blending

Base model:
1. All possible engineered features
2. No boxcox transformations, fill na with 0/'No'
3. No scaling, all ordinal transform
4. No outlier removal
5. XGBoost only

Unilaterally change the following and Record Test Score:
1. Removing engineered features
2. Boxcox transformation (with and without data leakage)
3. Scaling + Separate ordinal and One Hot Transformations
4. Outlier removal
5. Models blended

In [58]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, RobustScaler, PowerTransformer, FunctionTransformer
from sklearn.linear_model import Ridge, Lasso, RidgeCV, LassoCV
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer,root_mean_squared_error

In [41]:
train_path = './data/train.csv'
df = pd.read_csv(train_path)
X, Y = df.drop(labels=['SalePrice'], axis=1), np.log1p(df['SalePrice'])

In [42]:
# Fill NA

numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
def fillNull(X):
    for col in X.columns:
        if X[col].dtype in numeric_dtypes: X[col] = X[col].fillna(0)
        else: X[col] = X[col].fillna('No')

fillNull(X)

In [43]:
# Feature Engineering (simply taken from other Kaggle submissions, goal is to see effectiveness)

def featureEngineering(X: pd.DataFrame):
    # Summed features
    X['YrBltAndRemod']=X['YearBuilt']+X['YearRemodAdd']
    X['TotalSF']=X['TotalBsmtSF'] + X['1stFlrSF'] + X['2ndFlrSF']
    X['Total_sqr_footage'] = (X['BsmtFinSF1'] + X['BsmtFinSF2'] +
                                    X['1stFlrSF'] + X['2ndFlrSF'])
    X['Total_Bathrooms'] = (X['FullBath'] + (0.5 * X['HalfBath']) +
                                X['BsmtFullBath'] + (0.5 * X['BsmtHalfBath']))
    X['Total_porch_sf'] = (X['OpenPorchSF'] + X['3SsnPorch'] +
                                X['EnclosedPorch'] + X['ScreenPorch'] +
                                X['WoodDeckSF'])

    # Has features
    X['haspool'] = X['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
    X['has2ndfloor'] = X['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
    X['hasgarage'] = X['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
    X['hasbsmt'] = X['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
    X['hasfireplace'] = X['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

featureEngineering(X)

In [48]:
def find_model(modelTuple, params):
    categorical_cols = [col for col in X.columns if X[col].dtype not in numeric_dtypes]
    numerical_cols = [col for col in X.columns if X[col].dtype in numeric_dtypes]

    pipeline = Pipeline([
        ('encoder', ColumnTransformer([
            ('numerical_pipeline', Pipeline([
                ('add_constant', FunctionTransformer(lambda x: x + 1)),
                ('boxcox', PowerTransformer(method='box-cox')),
                ('scalar', RobustScaler())
            ]), numerical_cols),
        ('ohe', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
        ], remainder='passthrough')),
        
        modelTuple
    ])

    gridsearch = GridSearchCV(
        pipeline,
        scoring='neg_root_mean_squared_error',
        param_grid=params,
        n_jobs=-1,
        cv=5,
        return_train_score=True
    )

    gridsearch.fit(X, Y)

    results = pd.DataFrame(gridsearch.cv_results_['params'])
    results['train_rmse'] = gridsearch.cv_results_['mean_train_score']
    results['test_rmse'] = gridsearch.cv_results_['mean_test_score']
    results = results.sort_values(by='test_rmse', ascending=False)
    display(results.head())

    return gridsearch.best_estimator_

In [49]:
# XGB
xgb_tuple = ('xgb', xgb.XGBRegressor(random_state=42))

params_xgb = {
    'xgb__n_estimators': [100,500,3000],
    'xgb__learning_rate': [0.005, 0.01, 0.1],
    'xgb__max_depth': [3]
    }

xgb_model = find_model(xgb_tuple, params_xgb)

,xgb__learning_rate,xgb__max_depth,xgb__n_estimators,train_rmse,test_rmse
5,0.010,3,3000,-0.052027,-0.122756
2,0.005,3,3000,-0.070398,-0.123198
8,0.100,3,3000,-0.001815,-0.124345
7,0.100,3,500,-0.035238,-0.125069
6,0.100,3,100,-0.080298,-0.125783


In [53]:
# Ridge Regression
ridge_tuple = ('ridge', Ridge())

params_ridge = {
    'ridge__alpha': [18,19,20,21,22],
    'ridge__solver': ['auto', 'lsqr', 'sparse_cg', 'sag']
}

ridge_model = find_model(ridge_tuple, params_ridge)

,ridge__alpha,ridge__solver,train_rmse,test_rmse
17,22,lsqr,-0.106208,-0.126340
13,21,lsqr,-0.105910,-0.126343
9,20,lsqr,-0.105602,-0.126343
12,21,auto,-0.105911,-0.126343
15,21,sag,-0.105911,-0.126343


In [56]:
# Lasso Regression
lasso_tuple = ('lasso', Lasso())

params_lasso = {
    'lasso__alpha': [0.001, 0.002, 0.003, 0.005, 0.01],
    'lasso__max_iter': [10000]
}

lasso_model = find_model(lasso_tuple, params_lasso)

,lasso__alpha,lasso__max_iter,train_rmse,test_rmse
0,0.001,10000,-0.111158,-0.123300
1,0.002,10000,-0.121253,-0.128977
2,0.003,10000,-0.126319,-0.132219
3,0.005,10000,-0.132193,-0.136236
4,0.010,10000,-0.139513,-0.142248


In [59]:
# Voting Regressor
voting_model = VotingRegressor(estimators=[('xgb', xgb_model), ('ridge', ridge_model), ('lasso', lasso_model)])
voting_model.fit(X, Y)

VotingRegressor(estimators=[('xgb',
                             Pipeline(steps=[('encoder',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('numerical_pipeline',
                                                                               Pipeline(steps=[('add_constant',
                                                                                                FunctionTransformer(func=<function find_model.<locals>.<lambda> at 0x138c9d940>)),
                                                                                               ('boxcox',
                                                                                                PowerTransformer(method='box-cox')),
                                                                                               ('scalar',
                                                                                                RobustScaler())]),
                                                                               ['Id',
                                                                                'MSSubClass',
                                                                                'LotFr...
                                                                                'LandSlope',
                                                                                'Neighborhood',
                                                                                'Condition1',
                                                                                'Condition2',
                                                                                'BldgType',
                                                                                'HouseStyle',
                                                                                'RoofStyle',
                                                                                'RoofMatl',
                                                                                'Exterior1st',
                                                                                'Exterior2nd',
                                                                                'MasVnrType',
                                                                                'ExterQual',
                                                                                'ExterCond',
                                                                                'Foundation',
                                                                                'BsmtQual',
                                                                                'BsmtCond',
                                                                                'BsmtExposure',
                                                                                'BsmtFinType1',
                                                                                'BsmtFinType2',
                                                                                'Heating',
                                                                                'HeatingQC',
                                                                                'CentralAir',
                                                                                'Electrical', ...])])),
                                             ('lasso',
                                              Lasso(alpha=0.001,
                                                    max_iter=10000))]))])

In [68]:
# Stacking Regressor
stacking_model = StackingRegressor(estimators=[('xgb', xgb_model), ('ridge', ridge_model), ('lasso', lasso_model)])
stacking_model.fit(X, Y)

StackingRegressor(estimators=[('xgb',
                               Pipeline(steps=[('encoder',
                                                ColumnTransformer(remainder='passthrough',
                                                                  transformers=[('numerical_pipeline',
                                                                                 Pipeline(steps=[('add_constant',
                                                                                                  FunctionTransformer(func=<function find_model.<locals>.<lambda> at 0x138c9d940>)),
                                                                                                 ('boxcox',
                                                                                                  PowerTransformer(method='box-cox')),
                                                                                                 ('scalar',
                                                                                                  RobustScaler())]),
                                                                                 ['Id',
                                                                                  'MSSubClass',
                                                                                  'Lot...
                                                                                  'LandSlope',
                                                                                  'Neighborhood',
                                                                                  'Condition1',
                                                                                  'Condition2',
                                                                                  'BldgType',
                                                                                  'HouseStyle',
                                                                                  'RoofStyle',
                                                                                  'RoofMatl',
                                                                                  'Exterior1st',
                                                                                  'Exterior2nd',
                                                                                  'MasVnrType',
                                                                                  'ExterQual',
                                                                                  'ExterCond',
                                                                                  'Foundation',
                                                                                  'BsmtQual',
                                                                                  'BsmtCond',
                                                                                  'BsmtExposure',
                                                                                  'BsmtFinType1',
                                                                                  'BsmtFinType2',
                                                                                  'Heating',
                                                                                  'HeatingQC',
                                                                                  'CentralAir',
                                                                                  'Electrical', ...])])),
                                               ('lasso',
                                                Lasso(alpha=0.001,
                                                      max_iter=10000))]))])

In [69]:
def test_model(model):
    test_path = './data/test.csv'
    test_df = pd.read_csv(test_path)
    fillNull(test_df)
    featureEngineering(test_df)

    test_y = model.predict(test_df)

    test_df['SalePrice'] = np.expm1(test_y)
    test_df[['Id', 'SalePrice']].to_csv('results.csv', index=False)

    solution_path = './data/solution.csv'
    solution_df = pd.read_csv(solution_path)
    score = root_mean_squared_error(np.log(test_df['SalePrice']), np.log(solution_df['SalePrice']))
    return score

display(test_model(xgb_model))
display(test_model(ridge_model))
display(test_model(lasso_model))
display(test_model(voting_model))
display(test_model(stacking_model))

0.1336174285278018

0.1232598636666538

0.12365814280637331

0.12100262200598928

0.12321194939287175

# Results
Feature Engineering:
1. No feature engineering: 0.1345
2. Summed features: 0.1307
3. Has features: 0.1345

Scaling:
1. Baseline ordinal encoding:  0.1307
2. One hot encoding (instead of ordinal): 0.1317
3. One hot + Robust scalar: 0.1309

Models:
1. XGB: 0.1309
2. Ridge regression: 0.134
3. Lasso regression: 0.134

Transformation:
1. Boxcox Transformation (XGB 0.1336, Ridge 0.123, Lasso 0.123)

Ensembling Models:
1. Voting regressor 0.121
2. Stacking model: 0.123 w/o voting regressor, 0.121 with